# Gym opening in Paris

### Capstone Data Science Project
### Author : Alban STEFF

![gym](https://images.unsplash.com/photo-1571902943202-507ec2618e8f?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=968&q=80)

## Table of contents
- [Introduction](#intro)
- [Business problem](#problem)
- [Data](#data)
- [Methodology](#methodology)
  - [Define neighborhoods](#neighbors)
  - [Foursquare API : Get gym data](#api)
  - [Filter locations](#loc)
  - [Clustering data with K-means](#clustering)
- [Results](#results)
- [Discussion](#discussion)
- [Conclusion](#conclusion)

## Introduction <a name="intro"></a>

This project is part of the **Data Science Certification** created by the company **IBM** on the online courses platform **Coursera**.  
The **requirements** for this project are :
- Answer a question related to venues in a big city
- Use Foursquare API venues data
- Use k-means clustering algorithm

Before starting to run any code, make sure to have the following libraries installed in your working environnement !

In [0]:
pip install utm

In [0]:
import pandas as pd
import numpy as np
import requests
import folium
import utm
from folium.plugins import HeatMap, Fullscreen
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

As you can see, I am using the **'utm'** library for this project.  
It is a library to **convert latitude and longitude values into 2D coordinates.**  
The reason for this is that I needed to get distances between two specific points and working with coordinates is easier.  
To do so, I checked the **following map** to see if the **whole city of Paris was located in only one area** so I don't have any problems using UTM package.  
Since **the city is only inside one zone (31U)**, I can use coordinates instead of latitude and longitude to get distances between points.

![ParisUTM](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/LA2-Europe-UTM-zones.png/800px-LA2-Europe-UTM-zones.png)

## Business problem <a name="problem"></a>

In crowded cities, going to the **gym after your working days** can become a real **challenge** since a lot of people want to exercise at the **same time as you.**  

**It's not rare to wait** for some weights or machines **at peak hours** and this waiting time can transform in a **lack of motivation** to train in a gym during the weekdays.  

The goal of this project is to **find the best places in Paris to open a new gym** in order to reduce the amount of people going to one specific gym.

In fact, since some areas don't have any gym available, some workers are going to a gym relatively far from their workplace and with this project, we will try to find **which places can be of interest for gym companies**, so workers will stay around their working place to workout.

## Data <a name="data"></a>

For this project, data comes from three sources :
- Foursquare API to get venues data
- GeoPy data for longitude and latitude values
- Folium for interactive maps

To define candidates for clustering, I decided that I will not rely on Paris districts since they are too big for this analysis and they are not really relevant groups for gym facilities.  

You can find more details in the following section.

## Methodology <a name="methodology"></a>

This section is the **biggest part** of the report.  

It is divided in 4 sections :
- **Neighborhoods** : since I don't rely on Paris districts for the reason above, I have to create my own neighborhoods.
- **Foursquare API** : this is a great API to get venues location and I will use it to get sport venues in this project.
- **Filtering** : I will define some criteria to define a good location for a new gym and filter the sport data based on this definition to get final candidates.
- **Clustering** : k-means algorithm will cluster final candidates in a number of clusters that I will define later.

### Define neighborhoods <a name="neighbors"></a>

The first step for this analysis is to **create groups of the same size** and search for nearby **sport-related facilities.**  

To do so, I will first get the latitude and longitude value for the place which will define the **middle of Paris.**  

Let's say that **'Quai Saint-Michel'** is a good choice for the middle of Paris.

In [3]:
address = 'Quai Saint-Michel'
geolocator = Nominatim(user_agent="paris_explorer")
location = geolocator.geocode(address)
paris_latitude = location.latitude
paris_longitude = location.longitude
print('The geographical coordinate of the center of Paris are {}, {}.'.format(paris_latitude, paris_longitude))
print('The address for the center of Paris is : {}'.format(geolocator.reverse((paris_latitude, paris_longitude))))

The geographical coordinate of the center of Paris are 48.8533342, 2.3459126.
The address for the center of Paris is : Saint-Michel, Rue Xavier Privas, Îlot Saint-Séverin, Quartier de la Sorbonne, Paris, Île-de-France, France métropolitaine, 75005, France


Now, I need to **define two functions** to switch between latitudes, longitudes and 2D coordinates.  
*Why this transformation ?*  
**Folium (interactive map library)** is using latitude and longitude values to visualize the data while **distance calculation** is easier with 2D coordinates.

In [0]:
def to_latlon(x, y):
  # Zone variables for Paris
  zone_number = 31
  zone_letter = 'U'
  transform = utm.to_latlon(x, y, zone_number, zone_letter)
  return transform[0], transform[1]

def to_xy(latitude, longitude):
  transform = utm.from_latlon(latitude, longitude)
  return transform[0], transform[1]

Now, we are all set to get the **position of each group**, which will be used to find the **density of sport-related facilities.** Since our first goal for this analysis is to open a gym where we have almost no sport-related facilities, we can find the best areas to open a gym by **keeping geographical data with low density of gyms and dropping the rest.**

In [0]:
def get_groups():
  latitudes = []
  longitudes = []
  paris_x, paris_y = to_xy(paris_latitude, paris_longitude)

  # Define approximately the borders of Paris
  min_x = int(paris_x - 6000)
  max_x = int(paris_x + 5000)
  min_y = int(paris_y - 3500)
  max_y = int(paris_y + 5500)

  for x in range(min_x, max_x, 700):
    for y in range(min_y, max_y, 700):
      # Calculate distance from center
      # 6000 is approximately the radius of the 'Paris circle'
      #   -> Because Paris looks a little like a circle
      d = np.linalg.norm(np.array((x, y)) - np.array((paris_x, paris_y)))
      if d <= 6000:
        lat, lon = to_latlon(x, y)
        latitudes.append(lat)
        longitudes.append(lon)
  return latitudes, longitudes

In [6]:
lat_list, lon_list = get_groups()
print('Groups generated in Paris : {}'.format(len(lat_list)))

Groups generated in Paris : 185


Let's have a look at these groups on a **map of Paris**

In [7]:
map_paris = folium.Map([paris_latitude, paris_longitude], zoom_start = 12, tiles='Stamen Terrain')
folium.Marker([paris_latitude, paris_longitude], popup = 'Paris', icon = folium.Icon(color= 'green',icon_color='#006666',icon='info-sign')).add_to(map_paris)
for lat, lon in zip(lat_list, lon_list):
    folium.Circle([lat, lon], radius = 350, color = '#006666', fill = False).add_to(map_paris)
map_paris.save('neighbors.html')
map_paris

## Foursquare API : Get gym data <a name="api"></a>

Foursquare API has a **free version** which allows you to get **venues data and categories.**  
To use it, we need to **create an account** and copy our **credentials** in the cell below.  
If you want to run the following cells, you have to use your own credentials.

In [0]:
CLIENT_ID = ''
CLIENT_SECRET = ''

Here we define the API version, searching radius and venues limit (max: 100)

In [0]:
version = '20180605'
limit = 100
radius = 350

On [this](https://developer.foursquare.com/docs/build-with-foursquare/categories/) website, we can find **venue categories.**  
I decided to get the **sport-related facilities**.

In [0]:
sport_category = '4bf58dd8d48988d175941735'

Now, let's define a function to **store information in a dataframe** for all sport-related places.

In [0]:
def get_nearby_venues(latitudes, longitudes):
    
    venues_list = []
    for lat, lng in zip(latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, version, lat, lng, sport_category, radius, limit)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [0]:
paris_data = get_nearby_venues(lat_list, lon_list)

Let's check our sport data :

In [13]:
paris_data.tail()

,Unnamed: 0,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
520,520,48.859962,2.407176,Bien être à Paris,48.860351,2.404851,Yoga Studio
521,521,48.859962,2.407176,Paris Sport Club,48.861362,2.405883,Track
522,522,48.866259,2.407101,Momo & Cie,48.866487,2.403569,Track
523,523,48.878852,2.406952,Centre Sportif REM-GYM,48.876949,2.403459,Gym / Fitness Center
524,524,48.885149,2.406878,Energy Forme,48.887384,2.407057,Gym


Let's **drop duplicates** by checking latitude values which are **very sensitive**, meaning it's almost impossible to have the same value for two different places.  
But **names can be identical** so we need to be careful with dropping data by name.

In [14]:
paris_data.shape

(525, 7)

In [0]:
paris_venues = paris_data.drop_duplicates('Venue Latitude')

In [16]:
paris_venues.shape

(525, 7)

Since we defined a searching radius of 350 and a neighborhood radius of 350 too, it is good to see that we have no duplicates. We chose the right value to avoid duplicates.

Let's have a look at these unique facilities on a map :

In [17]:
map_paris = folium.Map([paris_latitude, paris_longitude], zoom_start = 12, tiles='Stamen Terrain')

for lat, lng, label in zip(paris_venues['Venue Latitude'], paris_venues['Venue Longitude'], paris_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='#00000000',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
map_paris.save('venues.html') 
map_paris

We have quite a lot sport facilities, so let's focus on **sub-categories** now.  
I will keep only the following options :

In [0]:
options = ['Gym', 'Gymnastics Gym', 'Outdoor Gym', 'Gym / Fitness Center']

Now, let's look at them on a map :

In [19]:
map_venues = folium.Map([paris_latitude, paris_longitude], zoom_start = 12, tiles='Stamen Terrain')

for lat, lng, cat, label in zip(paris_venues['Venue Latitude'], paris_venues['Venue Longitude'], paris_venues['Venue Category'], paris_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    venue_color = 'blue'
    if cat in options:
      venue_color = 'green'
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='#00000000',
        fill=True,
        fill_color=venue_color,
        fill_opacity=0.7,
        parse_html=False).add_to(map_venues)
map_venues.save('both_venues.html')
map_venues

Green points are gyms and blue points or other sport venues.  
It looks like there are more gyms than other types of sport facilities. But it's always better to do a calculation instead of just looking at the map :

In [20]:
gym_venues = paris_venues[paris_venues['Venue Category'].isin(options)].reset_index(drop = True)
gym_venues.tail()

,Unnamed: 0,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
349,517,48.853665,2.407250,Marza Nathalie,48.851142,2.405229,Gym
350,518,48.859962,2.407176,Les Cercles De La Forme Bagnolet,48.861375,2.405585,Gym / Fitness Center
351,519,48.859962,2.407176,Gym Suédoise Orteaux,48.856894,2.407987,Gym
352,523,48.878852,2.406952,Centre Sportif REM-GYM,48.876949,2.403459,Gym / Fitness Center
353,524,48.885149,2.406878,Energy Forme,48.887384,2.407057,Gym


In [21]:
print('Percentage of gyms : {:.2f} %'.format(gym_venues['Venue'].size / paris_venues['Venue'].size * 100))

Percentage of gyms : 67.43 %


**Most of our venues are gyms**, it's nice to know that to define our criteria later.  
To understand better the data, **heatmaps** are a great tool to **visualize the density of each venue.**  
You can find below a heatmap for all sport venues and then a map for gym venues only.

In [0]:
facilities_position = [[lat, lon] for lat, lon in zip(paris_venues['Venue Latitude'], paris_venues['Venue Longitude'])]
gym_position = [[lat, lon] for lat, lon in zip(gym_venues['Venue Latitude'], gym_venues['Venue Longitude'])]

### Heatmap for sport *venues*

In [23]:
map_sport = folium.Map([paris_latitude, paris_longitude], zoom_start = 13, tiles='Stamen Terrain')
HeatMap(facilities_position).add_to(map_sport)
Fullscreen().add_to(map_sport)
map_sport.save('sportHM.html')
map_sport

### Heatmap for gym venues

In [24]:
map_gyms = folium.Map([paris_latitude, paris_longitude], zoom_start = 13, tiles='Stamen Terrain')
HeatMap(gym_position).add_to(map_gyms)
Fullscreen().add_to(map_gyms)
map_gyms.save('gyms.html')
map_gyms

The maps show that we can find some areas with a **low density of gyms** like 'Quartier latin', 'Alésia', 'Bercy' or 'Invalides' for example.
Let's now focus on how to find **best places for a new gym !**

## Filter locations <a name="loc"></a>

Our data looks fine, we can now **define the criteria for a good gym opening !**  
The location must satisfy the problem at hand : allow workers to do their training near their workplace in good conditions.  
We can define a **good location** like this :  
- No sport facility within 100m
- No gym within 200m
- Less than 4 sport facilities in the neighborhood

To do this, we need to get back to **lat_list and lon_list** since our potential candidates are each group/neighborhood and not venues already existing.

Let's start with **distance to closer gym and venue**.

In [0]:
def get_dist(x_target, y_target, df):
  options = ['Gym', 'Gymnastics Gym', 'Outdoor Gym', 'Gym / Fitness Center']
  x_venues = []
  y_venues = []
  for lat, lon in zip(df['Venue Latitude'], df['Venue Longitude']):
    x, y = to_xy(lat, lon)
    x_venues.append(x)
    y_venues.append(y)
  df['x_venues'] = x_venues
  df['y_venues'] = y_venues
  closer_gym = 999999
  closer_venue = 999999
  for i in range(df['x_venues'].size):
    dist = np.linalg.norm(np.array((x_target, y_target)) - np.array((df['x_venues'][i], df['y_venues'][i])))
    if df['Venue Category'][i] in options:
      if dist < closer_gym:
        closer_gym = dist
    else:
      if dist < closer_venue:
        closer_venue = dist
  if closer_gym < closer_venue:
    closer_venue = closer_gym
  return closer_gym, closer_venue

In [0]:
def get_dist_frame(venues_df, neighs_lat, neighs_lon):
  # Convert lat, lon to x, y for neighborhoods
  x_neigh = []
  y_neigh = []
  for lat, lon in zip(neighs_lat, neighs_lon):
    x, y = to_xy(lat, lon)
    x_neigh.append(x)
    y_neigh.append(y)

  # List of distances to closer gym and facility for each neighborhood
  dist_frame = pd.DataFrame({'latitude': neighs_lat, 'longitude': neighs_lon, 'x': x_neigh, 'y': y_neigh, 'closer_gym': '?', 'closer_venue': '?'})
  for i in range(0, dist_frame['latitude'].size):
    closer_gym, closer_venue = get_dist(dist_frame['x'][i], dist_frame['y'][i], venues_df)
    dist_frame['closer_gym'][i] = closer_gym
    dist_frame['closer_venue'][i] = closer_venue
  return dist_frame

In [0]:
dist_df = get_dist_frame(paris_venues, lat_list, lon_list)

In [28]:
dist_df.tail(20)

,latitude,longitude,x,y,closer_gym,closer_venue
165,48.841022,2.397859,455817.999228,5.409958e+06,349.559,293.768
166,48.847319,2.397784,455817.999228,5.410658e+06,119.86,119.86
167,48.853616,2.397708,455817.999228,5.411358e+06,77.9903,77.9903
168,48.859912,2.397633,455817.999228,5.412058e+06,277.129,245.518
169,48.866209,2.397557,455817.999229,5.412758e+06,218.313,218.313
170,48.872506,2.397481,455817.999229,5.413458e+06,259.94,223.294
171,48.878803,2.397406,455817.999229,5.414158e+06,242.529,145.754
172,48.885099,2.397330,455817.999229,5.414858e+06,624.508,594.684
173,48.891396,2.397254,455817.999229,5.415558e+06,216.121,182.092
174,48.822181,2.407621,456517.999263,5.407858e+06,516.179,516.179


Finally, let's add **the count of facilities** in each group.

In [29]:
count = paris_data.groupby('Neighborhood Latitude').count()['Venue'].to_frame()
count['latitude'] = count.index
count.index = range(count['Venue'].size)
count.rename(columns={'Venue': 'count'}, inplace = True)
count.head()

,count,latitude
0,1,48.821536
1,2,48.821594
2,2,48.821652
3,3,48.821708
4,3,48.821764


Let's store all of this in a dataframe

In [30]:
neigh_df = pd.merge(dist_df, count, on = 'latitude')
neigh_df.head()

,latitude,longitude,x,y,closer_gym,closer_venue,count
0,48.827714,2.274030,446717.998645,5.408558e+06,187.883,187.883,3
1,48.852901,2.273666,446717.998646,5.411358e+06,286.124,137.155,3
2,48.859197,2.273575,446717.998646,5.412058e+06,236.334,114.82,6
3,48.834071,2.283477,447417.998698,5.409258e+06,289.238,132.72,4
4,48.859257,2.283117,447417.998699,5.412058e+06,124.223,89.2661,5


Let's extract information from this dataframe :

In [31]:
print('Max distance to gym : {}'.format(neigh_df['closer_gym'].max()))
print('Average distance to gym : {}'.format(neigh_df['closer_gym'].mean()))
print('Max distance to other sport venue : {}'.format(neigh_df['closer_venue'].max()))
print('Average distance to other sport venue : {}'.format(neigh_df['closer_venue'].mean()))
print('Average number of venues : {}'.format(neigh_df['count'].mean()))

Max distance to gym : 748.449802266426
Average distance to gym : 213.16096239478233
Max distance to other sport venue : 343.41166417452035
Average distance to other sport venue : 171.14064051054243
Average number of venues : 3.3214285714285716


We can create a new dataframe with the filtered data, corresponding to the rows **satisfying the requirements** for a potential new gym location.

In [0]:
conditions = (neigh_df['count'] <= 4) & (neigh_df['closer_gym'] >= 200) & (neigh_df['closer_venue'] >= 100)

In [33]:
paris_candidates = neigh_df[conditions].reset_index(drop = True)
paris_candidates.head()

,latitude,longitude,x,y,closer_gym,closer_venue,count
0,48.852901,2.273666,446717.998646,5.411358e+06,286.124,137.155,3
1,48.834071,2.283477,447417.998698,5.409258e+06,289.238,132.72,4
2,48.859316,2.292660,448117.998750,5.412058e+06,442.828,247.006,1
3,48.890799,2.292216,448117.998752,5.415558e+06,231.291,143.263,3
4,48.853078,2.302290,448817.998800,5.411358e+06,321.614,321.614,1


In [34]:
paris_candidates.shape

(33, 7)

We have 33 candidates for the final locations. It looks nice. Let's have a look at them on a map :

In [35]:
map_best = folium.Map([paris_latitude, paris_longitude], zoom_start = 12, tiles = 'Stamen Terrain')

for lat, lng in zip(paris_candidates['latitude'], paris_candidates['longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.6,
        parse_html=False).add_to(map_best)  
map_best.save('best.html')
map_best

## Clustering data with K-means <a name="clustering"></a>

**Machine learning** is useful now to cluster each candidate in a specific area.
Since we have **33 candidates**, we can **create 7 clusters** for them.  

Then, by getting the **cluster centers**, we will get the **final locations** for this project.

In [45]:
clusters = 7
k_means = KMeans(init = "k-means++", n_clusters = clusters, n_init = 12)
k_means.fit(paris_candidates[['x', 'y']].values)
centers = k_means.cluster_centers_
print(centers)

[[ 448817.99879818 5411941.33356995]
 [ 455257.99919681 5409538.00024215]
 [ 455677.99922118 5414018.0002437 ]
 [ 453017.99907084 5412058.00024094]
 [ 451757.99899381 5408698.00023887]
 [ 450392.99890629 5415558.00023931]
 [ 448817.9987975  5408791.33356911]]


In [0]:
final_lat = []
final_lon = []
for i in range(clusters):
  lat, lon = to_latlon(centers[i][0], centers[i][1])
  final_lat.append(lat)
  final_lon.append(lon)

In [47]:
map_final = folium.Map([paris_latitude, paris_longitude], zoom_start = 13, tiles = 'Stamen Terrain')

for lat, lng in zip(final_lat, final_lon):
    address = str(geolocator.reverse((lat, lng)))
    folium.Marker([lat, lng], popup = address, icon = folium.Icon(color= 'green',icon_color='red',icon='star')).add_to(map_final)
    folium.CircleMarker(
        [lat, lng],
        radius=20,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.4,
        parse_html=False).add_to(map_final)
map_final.save('Final.html')
map_final

Here we are ! We can now get the address for each point and share them with stakeholders :

In [48]:
for lat, lng in zip(final_lat, final_lon):
    print('Star address : {}'.format(geolocator.reverse((lat, lng))))

Star address : 131, Rue Saint-Dominique, Quartier du Gros-Caillou, Paris, Île-de-France, France métropolitaine, 75007, France
Star address : 33, Rue Proudhon, Quartier de Bercy, Paris, Île-de-France, France métropolitaine, 75012, France
Star address : Pharmacie Lafayette Daloy, 15, Rue Henri Ribière, Quartier d'Amérique, Paris, Île-de-France, France métropolitaine, 75019, France
Star address : Hôtel de Rohan, Rue Vieille du Temple, Quartier des Archives, Paris, Île-de-France, France métropolitaine, 75003, France
Star address : École Glacière, Rue Sœur Catherine Marie, Cité Florale, Quartier de la Maison-Blanche, Paris, Île-de-France, France métropolitaine, 75013, France
Star address : 39, Rue Lacroix, Quartier des Épinettes, Paris, Île-de-France, France métropolitaine, 75017, France
Star address : 135 bis, Rue Castagnary, Quartier Saint-Lambert, Paris, Île-de-France, France métropolitaine, 75015, France


## Results <a name="results"></a>

The final result for this project are the following star addresses :
- Star address : 131, Rue Saint-Dominique, Quartier du Gros-Caillou, Paris, Île-de-France, France métropolitaine, 75007, France
- Star address : 33, Rue Proudhon, Quartier de Bercy, Paris, Île-de-France, France métropolitaine, 75012, France
- Star address : Pharmacie Lafayette Daloy, 15, Rue Henri Ribière, Quartier d'Amérique, Paris, Île-de-France, France métropolitaine, 75019, France
- Star address : Hôtel de Rohan, Rue Vieille du Temple, Quartier des Archives, Paris, Île-de-France, France métropolitaine, 75003, France
- Star address : École Glacière, Rue Sœur Catherine Marie, Cité Florale, Quartier de la Maison-Blanche, Paris, Île-de-France, France métropolitaine, 75013, France
- Star address : 39, Rue Lacroix, Quartier des Épinettes, Paris, Île-de-France, France métropolitaine, 75017, France
- Star address : 135 bis, Rue Castagnary, Quartier Saint-Lambert, Paris, Île-de-France, France métropolitaine, 75015, France

## Discussion <a name="discussion"></a>

To find these results, I had to rely on my definition of what a good location means. I decided to take into consideration only 3 parameters : distance to closer gym, closer sport venue and total of venues in the area. Based on my criteria, I had 33 candidates but I could have more if I was less strict with the distances or the count of venues, although the final result would be less accurate for a new gym opening.  

The final addresses may have been found because of the capacity of neighborhoods in the border of Paris to satisfy the criteria since they have less neighbors than neighborhoods located in the center of Paris. That's why I decided to had the count idea in each neighborhood to diminish this effect.

Finally, each location is based on my calculations but opening a new gym near the Eiffel Tower for example might not be a good idea and also impossible due to the rental price here and regulations.

## Conclusion <a name="conclusion"></a>

To conclude this project, I recommend that stakeholders pay attention to the location of each final position based on what they want to do. Choosing between the 7 final locations must not be random.  

For instance, to target workers, opening a new gym in a crowded location at approximately 6pm can be better than just taking a random address that I found. Let's say that location 4 is well-known for hosting headquarters of companies, then this location is better to target workers than any other address.

This project is here to share recommendations but it's up to gym companies to find which address will best fit their needs.